### Name :  1-1-CisPdf2CsvByIndiv :
### Function :  Extract text content from CIS PDF files 
### Source Data :  高雄市議會1屆1次至2屆8次市政總質詢

In [1]:
# -*- coding: utf-8 -*-
"""
Created on May 13 2020
@author: johnson
"""

# Extract PDF by Page
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
import io
import glob
import csv
import os.path

###  user function : extract text data from PDF file via pdfminer 
定義第一個函式，extract_text_by_page，把pdf的文字抓出來

In [2]:
#定義把pdf轉文字的function
def extract_text_by_page(pdfFile):

    with open(pdfFile, 'rb') as fh:
        
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            
            resource_manager = PDFResourceManager()
            
            #codec = 'utf-8'
            #再系統裡面虛擬一個檔案處理
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            
            #  via TextConvert or HTMLConverter or XMLConverter
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            
            #開始抓出檔案內的文字
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            # print(text)
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()

### user function : 
###        1) receive PDF file
###        2) pass to  user function extract_text_by_page()   
###        3) write into output file

In [3]:
#傳三個參數:pdf_file(要處理的pdf), csv_path(輸出csv的檔案位子), isNew(是不是新的pdf)
def export_as_csv(pdf_file, csv_path, isNew):
    global FSeqNo #pdf的編號
    
    if isNew:
        opMode = "w"
    else:
        opMode = 'a'

    #把pdf讀進去    
    with open(csv_path, opMode, newline='', encoding='utf-8-sig') as csv_file:

        #存輸出csv的變數
        writer = csv.writer(csv_file)
        
        # 每處理一個pdf就把那個檔案的名字印出來
        print("PDF File: %s" % pdf_file)
        
        # 如果是新的一欄，就寫欄位
        # 會輸出成三個欄位: FileNo(檔案編號), FileName(檔案名稱), Text(pdf文字內容)
        if isNew:
            writer.writerow(['FileNo', 'FileName', 'Text'])   # write fields name into CSV file head
            
        pdfName = os.path.basename(pdf_file) #把pdf的名字存到pdfName這個變數裡面
        FSeqNo += 1
        contain = ""
        
        # pdf_file 是檔案路徑
        #呼叫剛剛定義的extract_text_by_page函式處理
        for page in extract_text_by_page(pdf_file):
            contain = contain + page 
            
        writer.writerow([FSeqNo, pdfName, contain])

### main 

下面這段開始處理資料，首先設定要讀入的檔案路徑，處理完後，會輸出一個叫做CisPdf2CsvbyIndiv的csv檔案

In [4]:
#設定要處理的pdf的所在路徑
#抓取某個資料夾底下所有某一類型(ex:csv、pdf)的檔案
pdf_path = "KCC Data/input_chen_fu/" 

# glob 會回傳資料夾裡面文件的檔案路徑
# {}裡面放的是上面pdf_path定義的位子
pdfs = glob.glob("{}/*.pdf".format(pdf_path))

#讀入pad的設定
newFlag = True #pdf是不是新的(一開始預設true是)
FSeqNo = 0 #設定pdf的編號(預設是0)

#設定要輸出的csv名字和路徑
#csv_path = "KCC Data/output/CisPdf2CsvbyIndiv.csv" 
csv_path = "KCC Data/output/chen_fu.csv" 

#用export_as_csv這個function輸出成csv檔案
for pdf in pdfs:
    export_as_csv(pdf, csv_path, newFlag)
    newFlag = False

PDF File: KCC Data/input_chen_fu\chen_fu.pdf


In [7]:
import pandas as pd 
#加 header=None 是要和pyhon說這個csv沒有欄位標頭
#記得要用encoding='utf-8'讀檔，不然會有亂碼，
#df = pd.read_csv("CisPdf2CsvByIndivSeg.csv", sep='delimiter', header=None, encoding='utf-8') #匯資料

df2 = pd.read_csv("KCC Data/output/chen_fu.csv",encoding='utf-8') #匯資料
df2

,FileNo,FileName,Text
0,1,chen_fu.pdf,165 立法院公報 第107卷 第93期 委員會紀錄 十一條之一及第四十三條條文修正草案...
